In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Replace this with your Spark Master URL
spark_master_url = "spark://192.168.18.110:7077"

spark = SparkSession.builder \
    .master(spark_master_url) \
    .appName("Read CSV from MinIO bucket") \
    .config("spark.hadoop.fs.s3a.access.key", "GD5Lg2x7tLaGxShjozFw") \
    .config("spark.hadoop.fs.s3a.secret.key", "H7J3mn7pI6oK35Xpuyrv6Lk4AsgymaSiQ2zwdAlu") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://192.168.18.115:9000") \
    .getOrCreate()

# Access the SparkContext from the SparkSession
sc = spark.sparkContext

# Check if the application is connected to the Spark master
def check_spark_connection():
    try:
        # Access the master URL and application ID
        master_url = sc.master
        app_id = sc.applicationId
        
        print(f"Connected to Spark Master at: {master_url}")
        print(f"Application ID: {app_id}")
        return True
    except Exception as e:
        print("Failed to connect to the Spark cluster.")
        print(f"Error: {e}")
        return False

# Run the connection check
if check_spark_connection():
    print("Spark connection successful.")
else:
    print("Failed to connect to Spark. Exiting...")
    spark.stop()  # Stop the Spark session if connection fails
    exit(1)

Connected to Spark Master at: spark://192.168.18.110:7077
Application ID: app-20241124013032-0009
Spark connection successful.


In [2]:
# Specify the S3 file path
s3_csv_path = "s3a://spark/allsales.csv"

# Read the CSV file
df = spark.read.csv(s3_csv_path, header=True, inferSchema=True)

# Show the DataFrame
df.show()

Py4JJavaError: An error occurred while calling o35.csv.
: java.util.ServiceConfigurationError: org.apache.spark.sql.sources.DataSourceRegister: Provider org.apache.spark.sql.execution.datasources.parquet.GeoParquetFileFormat could not be instantiated
	at java.util.ServiceLoader.fail(ServiceLoader.java:232)
	at java.util.ServiceLoader.access$100(ServiceLoader.java:185)
	at java.util.ServiceLoader$LazyIterator.nextService(ServiceLoader.java:384)
	at java.util.ServiceLoader$LazyIterator.next(ServiceLoader.java:404)
	at java.util.ServiceLoader$1.next(ServiceLoader.java:480)
	at scala.collection.convert.Wrappers$JIteratorWrapper.next(Wrappers.scala:46)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.Abstract

In [ ]:
df.createOrReplaceTempView("sales_data")
result = spark.sql("SELECT * FROM sales_data WHERE amount > 100")
result.show()